In [ ]:
import os

import boto3
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [2]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

# Load cut scenes from movies

In [9]:
def load_one_movie_cut_scene(load_data_info_dir, movie, save_data_dir, bucket):
    movie_path = os.path.join(load_data_info_dir, movie)

    scenarios = sorted(os.listdir(movie_path), key=lambda x: int(x))
    for scenario in scenarios:
        scenes_file = os.path.join(movie_path, scenario, "scenes.txt")

        with open(scenes_file, "r") as f:
            lines = f.readlines()

        save_dir_path = os.path.join(
            save_data_dir,
            movie_path.replace(load_data_info_dir + "/", ""),
            scenario,
        )
        os.makedirs(save_dir_path, exist_ok=True)
        for k, line in enumerate(lines):
            s3_client.download_file(
                Bucket=bucket,
                Key=line.strip(),
                Filename=os.path.join(save_dir_path, f"scene{k}.mp4"),
            )

In [2]:
s3_client = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
).client(
    "s3",
    endpoint_url="https://695d5a9fc666e505559765690b5db7dd.eu.r2.cloudflarestorage.com",
)

In [3]:
load_data_info_dir = "./batch_generation_data"

all_movies = os.listdir(load_data_info_dir)

movie = [x for x in all_movies if "Jurassic World Dominion" in x][0]

movie

'Jurassic World Dominion (2022) [1080p] [BluRay] [5.1] [YTS.MX]'

In [4]:
# s3_client.download_file(
#     Bucket="hf-movies",
#     Key="cutscenes/movies-1/Jurassic World Dominion (2022) [1080p] [BluRay] [5.1] [YTS.MX]/Jurassic.World.Dominion.2022.1080p.BluRay.x264.AAC5.1-[YTS.MX].mp4/tmp9dt5tflk-Scene-0697.mp4",
#     Filename=os.path.join(save_data_dir, "test.mp4"),
# )

In [6]:
save_data_dir = "./loaded_data"
os.makedirs(save_data_dir, exist_ok=True)

In [7]:
load_one_movie_cut_scene(
    load_data_info_dir=load_data_info_dir,
    movie=movie,
    save_data_dir=save_data_dir,
    bucket="hf-movies",
)

# Load middle frame from each cut scene

In [3]:
def load_one_movie_frame(load_data_info_dir, movie, save_data_dir, bucket):
    movie_path = os.path.join(load_data_info_dir, movie)

    scenarios = sorted(os.listdir(movie_path), key=lambda x: int(x.split(".txt")[0]))
    for scenario in scenarios:
        scenes_file = os.path.join(movie_path, scenario)

        with open(scenes_file, "r") as f:
            lines = f.readlines()

        save_dir_path = os.path.join(
            save_data_dir,
            movie_path.split("/")[-1],
            scenario.split(".txt")[0],
        )
        os.makedirs(save_dir_path, exist_ok=True)

        for line in lines:
            s3_client.download_file(
                Bucket=bucket,
                Key=line.strip(),
                Filename=os.path.join(
                    save_dir_path, "{:}".format(line.strip().split("/")[-1])
                ),
            )


In [4]:
s3_client = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
).client(
    "s3",
    endpoint_url="https://695d5a9fc666e505559765690b5db7dd.eu.r2.cloudflarestorage.com",
)

bucket = "hf-batch-generation-movies"

In [5]:
# s3_client.download_file(
#     Bucket=bucket,
#     Key="batch_generation_data/The Meg (2018) [WEBRip] [1080p] [YTS.AM]/3/images/0025.png",
#     Filename="test_load.png",
# )

In [5]:
load_data_info_dir = "./settings"

all_movies = os.listdir(load_data_info_dir)

# movie = [x for x in all_movies if "Jurassic World Dominion" in x][0]
# movie = "Avengers Age of Ultron (2015) [1080p]"
# movie = "The Meg (2018) [WEBRip] [1080p] [YTS.AM]"
movie = "Annabelle Comes Home (2019) [BluRay] [1080p] [YTS.LT]"
# movie = "The.Croods.A.New.Age.2020.1080p.WEB-DL.DD5.1.H.264-EVO[TGx]"

movie

'Annabelle Comes Home (2019) [BluRay] [1080p] [YTS.LT]'

In [6]:
save_data_dir = "./loaded_data"
os.makedirs(save_data_dir, exist_ok=True)

In [7]:
load_one_movie_frame(
    load_data_info_dir=load_data_info_dir,
    movie=movie,
    save_data_dir=save_data_dir,
    bucket=bucket,
)